<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Library
## Example: STILT, create a simple animation

This example shows how to search for STILT stations, load the data and create some simple visualisations and animations.

## Documentation
Full documentation for the library on the [project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on [pypi.org](https://pypi.org/project/icoscp/), source is available on [github](https://github.com/ICOS-Carbon-Portal/pylib)

## Load libraries

In [ ]:
from icoscp.stilt import stiltstation
import numpy as np
from IPython.display import HTML, display
from matplotlib import pyplot as plt, animation
# Display footprints in logarithmic scale
from matplotlib.colors import LogNorm

import cartopy
cartopy.config['data_dir'] = '/data/project/cartopy/'
import cartopy.crs as ccrs

# Import modules for spatiotemporal visualization:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import geoviews.tile_sources as gvts
from cartopy import crs
gv.extension('bokeh')

# A technical note: Due to an upcoming change of the package shapely 
# we suppress warnings they deliver in ShapelyDeprecationWarning
# (see the example of the last code cell of this notebook)
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore",category=ShapelyDeprecationWarning)

## Create STILT station object
Remember, that station get always returns a LIST of stilt stations objects

In [ ]:
st = stiltstation.get(id='KIT030')
print(st)

## Load footprints
The .get_fp function returns by default an xarray object

In [ ]:
start = '2018-01-01'
end = '2018-01-31'

stilt_fp = st.get_fp(start, end)

#View fp xarray:
stilt_fp

## Create 'images' with matplotlib

In [ ]:
# extract the footprints where a value is calculated....
fp = stilt_fp.foot.where(stilt_fp.foot>0)

In [ ]:
# just to see if it works, we create the first three images
# display footprints in logarithmic scale

for i in range(3):
    fp[i,:,:].plot(figsize=(4,3),norm=LogNorm())
    plt.grid(True)
    plt.show()
    plt.close()

## Create the figures for all footprints

In [ ]:
# Get a handle on the figure and the axes
fig, ax = plt.subplots(figsize=(8,4),subplot_kw=dict(projection=ccrs.PlateCarree()))

# Plot the initial frame. 

cax = fp[0,:,:].plot(    
    add_colorbar=True,
    cmap='GnBu',
    vmin=1.e-5,
    #vmin=np.min(fp).values,
    vmax=np.max(fp).values,
    cbar_kwargs={'extend':'neither'},
    norm=LogNorm()
)
gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
ax.coastlines('50m')
#define map extent to display only model domain
map_extent = [stilt_fp.lon.min(),stilt_fp.lon.max(),stilt_fp.lat.min(),stilt_fp.lat.max()]
ax.set_extent(map_extent,crs=ccrs.PlateCarree())

# Next we need to create a function that updates the values for the colormesh, as well as the title.
def animate(frame):
    cax.set_array(fp[frame,:,:].values.flatten())
    ax.set_title("Time = " + str(fp.coords['time'].values[frame])[:13])
    

# Finally, we use the animation module to create the animation.
ani = animation.FuncAnimation(
    fig,                  # figure
    animate,              # name of the function above
    frames=len(fp),       # Could also be iterable or list
    interval=200          # ms between frames
)

plt.close(fig)

## Animation with python built in libraries

Note: Based on how many footprints you have `len(fp)` it can take a while to create the animation, and you may possibly run out of memory. <br>
For this example we have selected footprints for a month for all time slots (every 3h),<br> hence we need to **calculate 240 images**, which requires approximately **1 minute.....be patient**.

In [ ]:
HTML(ani.to_jshtml())

### Sources
https://climate-cms.org/2019/09/03/python-animation.html

## Animation with Holoview & Geoview

In [ ]:
#Function to extract max absolute value from a tuple:
def return_max(val1, val2):
    
    if(abs(val1)>=abs(val2)):
        return abs(val1)
    
    else:
        return abs(val2)
    

#Set dimensions:
kdims = ['time', 'lon', 'lat']
vdims = ['foot']

#Get a tuple with min & max values for selected variable:
var_range = gv.Dataset(stilt_fp.foot.where(stilt_fp.foot>0.0, drop=True),
                       kdims=kdims, vdims=vdims).range('foot')

#Create geoviews dataset obj:
xr_dataset = gv.Dataset(stilt_fp.foot.where(stilt_fp.foot>0.0, drop=True),
                        kdims=kdims, vdims=vdims).redim.range(foot=(min(var_range),
                                                                    return_max(var_range[0],
                                                                               var_range[1])))

#Format dates:
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%dT%H:%M:%S'

#Set location of slider:
hv.output(widget_location='bottom')

In [ ]:
%%opts Image [logz=True]

# Note: Here is the code that gives deprecation warnings as mentioned in the beginning
gvpoints = gv.Points([(st.lon,st.lat)]).opts(color='darkred',width=500)
xr_dataset.to(gv.Image, ['lon', 'lat'], dynamic=True).opts(cmap='viridis',colorbar=True, alpha=0.5, #cmap='gist_heat_r'
                                                           width=600, height=500, 
                                                           tools=['hover'],)*gvts.CartoLight*gvpoints

### Sources
http://xarray.pydata.org/en/stable/io.html <br>
https://poopcode.com/coronavirus-covid-19-live-tracking-dashboard-death-counts/ <br>
https://stackoverflow.com/questions/17170229/setting-transparency-based-on-pixel-values-in-matplotlib <br>
https://data-dive.com/interactive-maps-made-easy-geoviews <br>